In [1]:
import pandas as pd 
import numpy as np  
import random
from sklearn.model_selection import train_test_split  
import tensorflow as tf 


In [2]:
# Data preparation
path = '/Users/Kassi/Desktop/voice-gender/voice.csv'
voice_data = pd.read_csv(path)
voice_data = voice_data.values

In [3]:
# There are multiple steps we will take in this section to transform the original data into format which we can easily plug inside tensorflow's tensors.
# Creating 1-hot vector from the original labels
# Randomly shuffle data
# Create train/test datasets

voices = voice_data[:, :-1] 
labels = voice_data[:, -1:]

# Classification: label(gender) = {male, female} → [1.0, 0.0], [0.0, 1.0]  
labels_tmp = []  
for label in labels:  
    tmp = []  
    if label[0] == 'male':  
        tmp = [1.0, 0.0]  
    else:   
        tmp = [0.0, 1.0]  
    labels_tmp.append(tmp)  
labels = np.array(labels_tmp) 

In [4]:
# randomly shuffle data
voices_tmp = []  
lables_tmp = []  
index_shuf = range(len(voices)) 
random.shuffle(index_shuf) 
for i in index_shuf:  
    voices_tmp.append(voices[i])  
    lables_tmp.append(labels[i])  
voices = np.array(voices_tmp)  
labels = np.array(lables_tmp) 

In [5]:
train_x, test_x, train_y, test_y = train_test_split(voices, labels, test_size=0.2)  

In [6]:
# Now we are all set to get started with creating the tensorflow graph. 
# It is important to understand that after this step there is no actual computation being done by tensorflow. 
# It just creates a lazy graph according to the nodes we create in the neural_network method.

In [7]:
# Learning Parameters
rate   =   0.0010  # training rate
epochs =  2000     # number of full training cycles (I select this number based on the trend of cost)
banch_size  = 68   # number of data points to train per batch

In [8]:
# Network Parameters
n_hidden_1 = 512  # number of nodes in hidden layer 1
n_hidden_2 = 512  # number of nodes in hidden layer 2
n_input    = voices.shape[-1]  # 20
n_classes  = 2
n_samples  = len(voices)  # 460

In [9]:
X = tf.placeholder('float', [None,n_input])
Y = tf.placeholder('float', [None,n_classes])

In [10]:
weights = {
    'w1' : tf.Variable(tf.random_normal([n_input,    n_hidden_1])),
    'w2' : tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, n_classes ]))
}

biases = {
    'b1' : tf.Variable(tf.random_normal([n_hidden_1])),
    'b2' : tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_classes ]))
}

In [11]:
def neural_network(X, weights, biases):  
    # Hidden Layer 1
    layer1 = tf.matmul(X, weights['w1'])
    layer1 = tf.add(layer1, biases['b1'])
    layer1 = tf.nn.softmax(layer1)
    
    # Hidden Layer 2
    layer2 = tf.matmul(layer1, weights['w2'])
    layer2 = tf.add(layer2, biases['b2'])
    layer2 = tf.nn.softmax(layer2)
    
    # Output Layer
    output = tf.matmul(layer2, weights['out'])
    output = tf.add(output, biases['out'])
    output = tf.nn.softmax(output)
    
    return output 

In [12]:
# In this section we define the optimizer and loss functions. 
# We also define our notion of correct and incorrect prediction and what we mean by accuracy.

def train_neural_network(train_x,train_y,test_x,test_y):
    model = neural_network(X, weights, biases)
    f_cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
    f_optimizer = tf.train.AdamOptimizer(learning_rate=rate).minimize(f_cost)
    with tf.Session() as s:
        s.run(tf.global_variables_initializer())
        for epoch in range(epochs):
            cost_avg = 0.
            batch_total = len(train_x) // banch_size
            for banch in range(batch_total):
                voice_banch = train_x[banch*banch_size:(banch+1)*(banch_size)]  
                label_banch = train_y[banch*banch_size:(banch+1)*(banch_size)]        
                _, cost = s.run([f_optimizer,f_cost], feed_dict={X:voice_banch, Y: label_banch})        
                cost_avg += cost / batch_total
        
            print('Epoch {}: cost={:.4f}'.format(epoch+1, cost_avg))
    
        # testing
        
        # This gives us a list of booleans
        prediction = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))  
        # We cast to floating point numbers and then take the mean
        accuracy = tf.reduce_mean(tf.cast(prediction, dtype=tf.float32))  
        accuracy = s.run(accuracy, feed_dict={X: train_x, Y: train_y})  
        print('In-sample accuracy in Neural Network: %s'  % (accuracy)) 
        
        accuracy = tf.reduce_mean(tf.cast(prediction, dtype=tf.float32)) 
        accuracy = s.run(accuracy, feed_dict={X: test_x, Y: test_y})  
        print('Out-of-sample accuracy in Neural Network: %s'  % (accuracy)) 
    


In [13]:
## Execute tf graph
# Now that the graph is set up, we can run it

print('Accuracy in Neural Network with all features:')
train_neural_network(train_x,train_y,test_x,test_y)

Accuracy in Neural Network with all features:
Epoch 1: cost=0.7043
Epoch 2: cost=0.7033
Epoch 3: cost=0.7023
Epoch 4: cost=0.7014
Epoch 5: cost=0.7006
Epoch 6: cost=0.6998
Epoch 7: cost=0.6991
Epoch 8: cost=0.6985
Epoch 9: cost=0.6979
Epoch 10: cost=0.6974
Epoch 11: cost=0.6969
Epoch 12: cost=0.6965
Epoch 13: cost=0.6961
Epoch 14: cost=0.6958
Epoch 15: cost=0.6954
Epoch 16: cost=0.6952
Epoch 17: cost=0.6949
Epoch 18: cost=0.6946
Epoch 19: cost=0.6944
Epoch 20: cost=0.6941
Epoch 21: cost=0.6939
Epoch 22: cost=0.6937
Epoch 23: cost=0.6935
Epoch 24: cost=0.6932
Epoch 25: cost=0.6930
Epoch 26: cost=0.6928
Epoch 27: cost=0.6925
Epoch 28: cost=0.6922
Epoch 29: cost=0.6918
Epoch 30: cost=0.6914
Epoch 31: cost=0.6910
Epoch 32: cost=0.6905
Epoch 33: cost=0.6899
Epoch 34: cost=0.6894
Epoch 35: cost=0.6889
Epoch 36: cost=0.6881
Epoch 37: cost=0.6876
Epoch 38: cost=0.6871
Epoch 39: cost=0.6866
Epoch 40: cost=0.6860
Epoch 41: cost=0.6854
Epoch 42: cost=0.6849
Epoch 43: cost=0.6843
Epoch 44: cost=0.

Epoch 367: cost=0.5499
Epoch 368: cost=0.5496
Epoch 369: cost=0.5493
Epoch 370: cost=0.5490
Epoch 371: cost=0.5487
Epoch 372: cost=0.5484
Epoch 373: cost=0.5482
Epoch 374: cost=0.5479
Epoch 375: cost=0.5476
Epoch 376: cost=0.5473
Epoch 377: cost=0.5470
Epoch 378: cost=0.5467
Epoch 379: cost=0.5464
Epoch 380: cost=0.5462
Epoch 381: cost=0.5459
Epoch 382: cost=0.5456
Epoch 383: cost=0.5453
Epoch 384: cost=0.5450
Epoch 385: cost=0.5447
Epoch 386: cost=0.5444
Epoch 387: cost=0.5442
Epoch 388: cost=0.5439
Epoch 389: cost=0.5436
Epoch 390: cost=0.5433
Epoch 391: cost=0.5430
Epoch 392: cost=0.5427
Epoch 393: cost=0.5424
Epoch 394: cost=0.5421
Epoch 395: cost=0.5418
Epoch 396: cost=0.5415
Epoch 397: cost=0.5412
Epoch 398: cost=0.5409
Epoch 399: cost=0.5406
Epoch 400: cost=0.5403
Epoch 401: cost=0.5400
Epoch 402: cost=0.5397
Epoch 403: cost=0.5394
Epoch 404: cost=0.5391
Epoch 405: cost=0.5388
Epoch 406: cost=0.5385
Epoch 407: cost=0.5382
Epoch 408: cost=0.5379
Epoch 409: cost=0.5376
Epoch 410: 

Epoch 724: cost=0.4645
Epoch 725: cost=0.4644
Epoch 726: cost=0.4642
Epoch 727: cost=0.4641
Epoch 728: cost=0.4639
Epoch 729: cost=0.4637
Epoch 730: cost=0.4636
Epoch 731: cost=0.4634
Epoch 732: cost=0.4632
Epoch 733: cost=0.4631
Epoch 734: cost=0.4629
Epoch 735: cost=0.4627
Epoch 736: cost=0.4626
Epoch 737: cost=0.4624
Epoch 738: cost=0.4623
Epoch 739: cost=0.4621
Epoch 740: cost=0.4619
Epoch 741: cost=0.4618
Epoch 742: cost=0.4616
Epoch 743: cost=0.4615
Epoch 744: cost=0.4613
Epoch 745: cost=0.4612
Epoch 746: cost=0.4610
Epoch 747: cost=0.4608
Epoch 748: cost=0.4607
Epoch 749: cost=0.4605
Epoch 750: cost=0.4604
Epoch 751: cost=0.4602
Epoch 752: cost=0.4601
Epoch 753: cost=0.4599
Epoch 754: cost=0.4597
Epoch 755: cost=0.4596
Epoch 756: cost=0.4594
Epoch 757: cost=0.4593
Epoch 758: cost=0.4591
Epoch 759: cost=0.4590
Epoch 760: cost=0.4588
Epoch 761: cost=0.4587
Epoch 762: cost=0.4585
Epoch 763: cost=0.4584
Epoch 764: cost=0.4582
Epoch 765: cost=0.4581
Epoch 766: cost=0.4579
Epoch 767: 

Epoch 1077: cost=0.4269
Epoch 1078: cost=0.4269
Epoch 1079: cost=0.4269
Epoch 1080: cost=0.4268
Epoch 1081: cost=0.4268
Epoch 1082: cost=0.4267
Epoch 1083: cost=0.4267
Epoch 1084: cost=0.4266
Epoch 1085: cost=0.4266
Epoch 1086: cost=0.4266
Epoch 1087: cost=0.4265
Epoch 1088: cost=0.4265
Epoch 1089: cost=0.4264
Epoch 1090: cost=0.4264
Epoch 1091: cost=0.4264
Epoch 1092: cost=0.4263
Epoch 1093: cost=0.4263
Epoch 1094: cost=0.4262
Epoch 1095: cost=0.4262
Epoch 1096: cost=0.4261
Epoch 1097: cost=0.4261
Epoch 1098: cost=0.4261
Epoch 1099: cost=0.4260
Epoch 1100: cost=0.4260
Epoch 1101: cost=0.4259
Epoch 1102: cost=0.4259
Epoch 1103: cost=0.4259
Epoch 1104: cost=0.4258
Epoch 1105: cost=0.4258
Epoch 1106: cost=0.4257
Epoch 1107: cost=0.4257
Epoch 1108: cost=0.4257
Epoch 1109: cost=0.4256
Epoch 1110: cost=0.4256
Epoch 1111: cost=0.4255
Epoch 1112: cost=0.4255
Epoch 1113: cost=0.4255
Epoch 1114: cost=0.4254
Epoch 1115: cost=0.4254
Epoch 1116: cost=0.4254
Epoch 1117: cost=0.4253
Epoch 1118: cost

Epoch 1420: cost=0.4135
Epoch 1421: cost=0.4132
Epoch 1422: cost=0.4135
Epoch 1423: cost=0.4131
Epoch 1424: cost=0.4134
Epoch 1425: cost=0.4131
Epoch 1426: cost=0.4133
Epoch 1427: cost=0.4130
Epoch 1428: cost=0.4133
Epoch 1429: cost=0.4130
Epoch 1430: cost=0.4132
Epoch 1431: cost=0.4129
Epoch 1432: cost=0.4132
Epoch 1433: cost=0.4129
Epoch 1434: cost=0.4131
Epoch 1435: cost=0.4128
Epoch 1436: cost=0.4130
Epoch 1437: cost=0.4127
Epoch 1438: cost=0.4130
Epoch 1439: cost=0.4127
Epoch 1440: cost=0.4129
Epoch 1441: cost=0.4126
Epoch 1442: cost=0.4129
Epoch 1443: cost=0.4126
Epoch 1444: cost=0.4128
Epoch 1445: cost=0.4125
Epoch 1446: cost=0.4128
Epoch 1447: cost=0.4125
Epoch 1448: cost=0.4127
Epoch 1449: cost=0.4124
Epoch 1450: cost=0.4127
Epoch 1451: cost=0.4124
Epoch 1452: cost=0.4126
Epoch 1453: cost=0.4123
Epoch 1454: cost=0.4125
Epoch 1455: cost=0.4122
Epoch 1456: cost=0.4125
Epoch 1457: cost=0.4122
Epoch 1458: cost=0.4124
Epoch 1459: cost=0.4121
Epoch 1460: cost=0.4124
Epoch 1461: cost

Epoch 1764: cost=0.4073
Epoch 1765: cost=0.4051
Epoch 1766: cost=0.4058
Epoch 1767: cost=0.4071
Epoch 1768: cost=0.4047
Epoch 1769: cost=0.4051
Epoch 1770: cost=0.4065
Epoch 1771: cost=0.4052
Epoch 1772: cost=0.4057
Epoch 1773: cost=0.4057
Epoch 1774: cost=0.4066
Epoch 1775: cost=0.4047
Epoch 1776: cost=0.4047
Epoch 1777: cost=0.4045
Epoch 1778: cost=0.4046
Epoch 1779: cost=0.4048
Epoch 1780: cost=0.4048
Epoch 1781: cost=0.4052
Epoch 1782: cost=0.4056
Epoch 1783: cost=0.4064
Epoch 1784: cost=0.4048
Epoch 1785: cost=0.4047
Epoch 1786: cost=0.4043
Epoch 1787: cost=0.4043
Epoch 1788: cost=0.4044
Epoch 1789: cost=0.4043
Epoch 1790: cost=0.4044
Epoch 1791: cost=0.4043
Epoch 1792: cost=0.4044
Epoch 1793: cost=0.4044
Epoch 1794: cost=0.4047
Epoch 1795: cost=0.4049
Epoch 1796: cost=0.4058
Epoch 1797: cost=0.4053
Epoch 1798: cost=0.4057
Epoch 1799: cost=0.4051
Epoch 1800: cost=0.4053
Epoch 1801: cost=0.4053
Epoch 1802: cost=0.4056
Epoch 1803: cost=0.4052
Epoch 1804: cost=0.4052
Epoch 1805: cost

We are wondering if there is a big difference in the results when used Keras deep learning library with a TensorFlow backend vs. our own step-by-step algorithm.

In [14]:
#Importing Libraries for ANN
import keras
from keras.models import Sequential
from keras.layers import Dense

#Initialisng the ANN
classifier = Sequential()

#Input Layer and First Hidden Layer
classifier.add(Dense(units = 512, activation = 'softmax', input_dim = train_x.shape[1]))

#Adding the Second hidden layer
classifier.add(Dense(units = 512, activation = 'softmax'))

# Adding the output layer
classifier.add(Dense(units = 2, activation = 'softmax'))

#Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Using TensorFlow backend.


In [15]:
#Fitting ANN to the training set
classifier.fit(train_x, train_y, batch_size = 68, epochs = 2000, verbose = 0)

In [16]:
# Predicting the Test set results
pred_train_y = classifier.predict(train_x)
pred_train_y = (pred_train_y > 0.5)
# pred_train_y

In [17]:
labels_tmp = []  
for label in pred_train_y:  
    tmp = []  
    if label[0] == True:  
        tmp = [1,0]  
    else:   
        tmp = [0,1]  
    labels_tmp.append(tmp)  
new_result = np.array(labels_tmp) 

In [18]:
from sklearn.metrics import accuracy_score
print('In-sample accuracy in Neural Network using Keras package (with all features):%s' % (accuracy_score(train_y, new_result)))

In-sample accuracy in Neural Network using Keras package (with all features):0.910326086957


In [19]:
# Predicting the Test set results
pred_test_y = classifier.predict(test_x)
pred_test_y = (pred_test_y > 0.5)
# pred_test_y

In [20]:
labels_tmp = []  
for label in pred_test_y:  
    tmp = []  
    if label[0] == True:  
        tmp = [1,0]  
    else:   
        tmp = [0,1]  
    labels_tmp.append(tmp)  
new_result = np.array(labels_tmp) 

In [21]:
print('Out-of-sample accuracy in Neural Network using Keras package (with all features):%s' % (accuracy_score(test_y, new_result)))

Out-of-sample accuracy in Neural Network using Keras package (with all features):0.836956521739


In [22]:
# Is it enough of these two features to make predictions? 
## Neural Network with 2 features ('meanfun', 'IQR')
train_x = train_x[:,[5,12]]
test_x = test_x[:,[5,12]]

In [23]:
train_x.shape[1]

2

In [24]:
# It seems we can better result when using Keras deep learning library, 
# so we will continue to use this package to do prediction.

#Initialisng the ANN
classifier = Sequential()

#Input Layer and First Hidden Layer
classifier.add(Dense(units = 512, activation = 'softmax', input_dim = train_x.shape[1]))

#Adding the Second hidden layer
classifier.add(Dense(units = 512, activation = 'softmax'))

# Adding the output layer
classifier.add(Dense(units = 2, activation = 'softmax'))

#Compiling the ANN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [25]:
#Fitting ANN to the training set
classifier.fit(train_x, train_y, batch_size = 68, epochs = 2000, verbose = 0)

In [26]:
# Predicting the Train set results
pred_y = classifier.predict(train_x)
pred_y = (pred_y > 0.5)
# pred_y

In [27]:
labels_tmp = []  
for label in pred_y:  
    tmp = []  
    if label[0] == True:  
        tmp = [1,0]  
    else:   
        tmp = [0,1]  
    labels_tmp.append(tmp)  
new_result = np.array(labels_tmp) 

In [28]:
print('In-sample accuracy in Neural Network using Keras package (with 2 features):%s' % (accuracy_score(train_y, new_result)))

In-sample accuracy in Neural Network using Keras package (with 2 features):0.885869565217


In [29]:
# Predicting the Test set results
pred_y = classifier.predict(test_x)
pred_y = (pred_y > 0.5)
# pred_y

In [30]:
labels_tmp = []  
for label in pred_y:  
    tmp = []  
    if label[0] == True:  
        tmp = [1,0]  
    else:   
        tmp = [0,1]  
    labels_tmp.append(tmp)  
new_result = np.array(labels_tmp) 

In [31]:
print('Out-of-sample accuracy in Neural Network using Keras package (with 2 features):%s' % (accuracy_score(test_y, new_result)))

Out-of-sample accuracy in Neural Network using Keras package (with 2 features):0.869565217391


In case with only 2 features(‘meanfun’, ‘IQR’) without feature scaling results are more precise. 

Neural Network with only two features gives 87.0% against 83.7% with all features. 